In [240]:
import matplotlib.pyplot as plt 
import pandas as pd
import math
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU 

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MaxAbsScaler

import psycopg2
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt 
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'
import seaborn as sns

import keras_functions
from keras_functions import *

In [2]:
con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

In [3]:
binvars = get_categorical_variables([
    'all_participanttypes',
    'all_tags',
    'all_language'
])

# Initial pipeline

In [4]:
with open('set_train.txt') as f:
    train_handles = [h.strip() for h in f.readlines()]
with open('set_test.txt') as f:
    test_handles = [h.strip() for h in f.readlines()]

In [346]:
# user columns can be out of order, use tourist's as correct order
with open('rnn_train/tourist.csv') as f:
    correct_cols = [t.strip() for t in f.readline().split(',')]

In [409]:
user_batch = 5#len(test_handles)

In [410]:
from os.path import exists
reload(keras_functions)
X = []
Y = []
maxt = 0
maxtimepts = 5000
lens = [0] * user_batch
runsum = 0
for i in range(user_batch):
        filename = 'rnn_train/' + train_handles[i] + ".csv"
        #if i%100 == 0:
        #print i, filename
        if not exists(filename):
            continue
        x, y, t, colnames = keras_functions.get_user_data(
            train_handles[i],
            binvars=binvars,
            month=3,
            maxtimepts=maxtimepts,
            columns=correct_cols)

        if x is None:
            continue
        maxt = max(t, maxt)
        X.append(x)
        Y.append(y)
        runsum += x.shape[0]
        lens[i] = runsum


0    -1
1    -1
2    -1
3    -1
4    -1
5    -1
6    -1
7    -1
8    -1
9    -1
10   -1
11   -1
12   -1
13   -1
14   -1
15   -1
16   -1
17   -1
18   -1
19   -1
20   -1
21   -1
22   -1
23   -1
24   -1
25   -1
26   -1
27   -1
28   -1
29   -1
30   -1
31   -1
32   -1
33   -1
34   -1
35   -1
Name: problem_rating, dtype: int64
29
0    -1
27   -1
23   -1
22   -1
21   -1
18   -1
16   -1
15   -1
14   -1
13   -1
12   -1
17   -1
10   -1
1    -1
11   -1
3    -1
4    -1
5    -1
2    -1
9    -1
6    -1
7    -1
8    -1
31   -1
30   -1
29   -1
28   -1
19   -1
26   -1
25   -1
24   -1
20   -1
Name: problem_rating, dtype: int64
29
32   -1
33   -1
34   -1
Name: problem_rating, dtype: int64
29
35   -1
Name: problem_rating, dtype: int64
0    -1
1    -1
2    -1
3    -1
4    -1
5    -1
6    -1
7    -1
8    -1
9    -1
10   -1
11   -1
12   -1
13   -1
14   -1
15   -1
16   -1
17   -1
18   -1
19   -1
20   -1
21   -1
22   -1
23   -1
24   -1
25   -1
26   -1
27   -1
28   -1
29   -1
     ..
38   -1
39   -1
40   -1
41 

Name: problem_rating, dtype: int64
29
382   -1
Name: problem_rating, dtype: int64
29
383   -1
384   -1
385   -1
387   -1
388   -1
Name: problem_rating, dtype: int64
29
386   -1
Name: problem_rating, dtype: int64
29
391   -1
392   -1
389   -1
390   -1
Name: problem_rating, dtype: int64
29
393   -1
394   -1
395   -1
Name: problem_rating, dtype: int64
29
402   -1
405   -1
404   -1
403   -1
400   -1
399   -1
398   -1
397   -1
396   -1
401   -1
Name: problem_rating, dtype: int64
29
406   -1
407   -1
408   -1
148   -1
Name: problem_rating, dtype: int64
29
409   -1
410   -1
411   -1
Name: problem_rating, dtype: int64
29
415   -1
413   -1
412   -1
Name: problem_rating, dtype: int64
29
416   -1
417   -1
418   -1
419   -1
414   -1
Name: problem_rating, dtype: int64
29
420   -1
421   -1
422   -1
423   -1
Name: problem_rating, dtype: int64
29
431   -1
430   -1
429   -1
427   -1
426   -1
424   -1
428   -1
453   -1
425   -1
Name: problem_rating, dtype: int64
29
432   -1
433   -1
Name: problem_rating

In [419]:
user = 'tourist'
data = pd.read_csv('rnn_train/%s.csv'%user)

In [416]:
correct_cols

['CHALLENGED',
 'COMPILATION_ERROR',
 'CONTESTANT',
 'Delphi',
 'FPC',
 'GNU C++',
 'GNU C++0x',
 'GNU C++11',
 'GNU C++14',
 'GYM',
 'IDLENESS_LIMIT_EXCEEDED',
 'Java 7',
 'Java 8',
 'MEMORY_LIMIT_EXCEEDED',
 'MS C++',
 'Mysterious Language',
 'OK',
 'OUT_OF_COMPETITION',
 'PHP',
 'PRACTICE',
 'PRESENTATION_ERROR',
 'RUNTIME_ERROR',
 'SKIPPED',
 'Secret_171',
 'TIME_LIMIT_EXCEEDED',
 'Tcl',
 'VIRTUAL',
 'WRONG_ANSWER',
 'contestid',
 'delta_smoothed_1months',
 'delta_smoothed_2months',
 'delta_smoothed_3months',
 'delta_smoothed_4months',
 'delta_smoothed_5months',
 'handle',
 'index',
 'newrating',
 'oldrating',
 'points',
 'problem_rating',
 'rank',
 'ratingupdatetimeseconds',
 'smoothed_1months',
 'smoothed_2months',
 'smoothed_3months',
 'smoothed_4months',
 'smoothed_5months',
 'solvetimeseconds',
 'starttimeseconds',
 'stoptimeseconds',
 'shortest paths',
 '2-sat',
 'greedy',
 'meet-in-the-middle',
 'matrices',
 'number theory',
 'constructive algorithms',
 'chinese remainder th

In [162]:
reload(keras_functions)
batchsize = [1, X[0].shape[1], X[0].shape[2]]
gru1 = 10
gru2 = 10
dense = 10
model = keras_functions.create_model([gru1, gru2, dense], batchsize)

In [163]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_6 (Masking)          (1, 5000, 110)            0         
_________________________________________________________________
gru_11 (GRU)                 (1, 5000, 10)             3630      
_________________________________________________________________
dropout_14 (Dropout)         (1, 5000, 10)             0         
_________________________________________________________________
gru_12 (GRU)                 (1, 10)                   630       
_________________________________________________________________
dense_9 (Dense)              (1, 1)                    11        
Total params: 4,271
Trainable params: 4,271
Non-trainable params: 0
_________________________________________________________________


# Scaling
RATING MUST BE ON THE SAME SCALE


In [398]:
Xflat = np.concatenate(X)
print Xflat.shape
Xflat = np.reshape(Xflat, [Xflat.shape[0]*Xflat.shape[1], Xflat.shape[2]])
print Xflat.shape
#scaler = MaxAbsScaler().fit(Xflat)

(10, 5000, 108)
(50000, 108)


In [404]:
xmax = np.max(Xflat, axis=0)
list(xmax)
xmax[colnames.index('smoothed_3months')]
xmax[colnames.index('problem_rating')]

0.0

In [272]:
Yflat = np.concatenate(Y)
scaler = MaxAbsScaler(200).fit(Yflat.reshape(-1,1))

In [273]:
scaler.transform([[-200], [200]])

array([[-1.69491525],
       [ 1.69491525]])

In [242]:
epochs = 1
for e in range(epochs):
    for s in range(len(X)):
        print s
        for b in range(X[s].shape[0]):
            x_in = scaler.transform(X[s][b])
            x_in = np.expand_dims(x_in, axis=0)
            y_in = np.expand_dims(Y[s][b], axis=0)
            history = model.fit(x_in, y_in, epochs=1, shuffle=False, batch_size=1)

0
Epoch 1/1


KeyboardInterrupt: 

In [239]:
testarray = [-1, -1, 10, 100, 1000]

/usr/local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:785: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:818: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


array([-0.001, -0.001,  0.01 ,  0.1  ,  1.   ])

# DEBUGGING 

In [97]:
list(colnames).index('hours_submit_to_solve')

111

In [298]:
colnames

['CONTESTANT',
 'GNU C++',
 'MEMORY_LIMIT_EXCEEDED',
 'OK',
 'PRACTICE',
 'RUNTIME_ERROR',
 'TIME_LIMIT_EXCEEDED',
 'VIRTUAL',
 'WRONG_ANSWER',
 'points',
 'problem_rating',
 'rank',
 'smoothed_3months',
 'shortest paths',
 '2-sat',
 'greedy',
 'meet-in-the-middle',
 'matrices',
 'number theory',
 'constructive algorithms',
 'chinese remainder theorem',
 'implementation',
 'ternary search',
 'schedules',
 'dsu',
 'dfs and similar',
 'graph matchings',
 'string suffix structures',
 'math',
 'probabilities',
 'fft',
 'divide and conquer',
 'two pointers',
 'trees',
 'data structures',
 'flows',
 'sortings',
 'expression parsing',
 'dp',
 'hashing',
 'bitmasks',
 '*special',
 'geometry',
 'combinatorics',
 'graphs',
 'brute force',
 'games',
 'binary search',
 'strings',
 'PRESENTATION_ERROR',
 'TESTING',
 'CRASHED',
 'CHALLENGED',
 'REJECTED',
 'FAILED',
 'SKIPPED',
 'IDLENESS_LIMIT_EXCEEDED',
 'COMPILATION_ERROR',
 'PARTIAL',
 'GYM',
 'OUT_OF_COMPETITION',
 'Scala',
 'Delphi',
 'JavaScr

In [108]:
list(X[1][0][:20, 107])

[240.27944444444444,
 243.03,
 265.55444444444447,
 219.19305555555556,
 264.39888888888891,
 270.01055555555558,
 157.60499999999999,
 241.48444444444445,
 156.24166666666667,
 195.29472222222222,
 266.0361111111111,
 241.34666666666666,
 242.24388888888888,
 218.71111111111111,
 219.02277777777778,
 269.77027777777778,
 -1.0,
 122.39583333333333,
 222.07166666666666,
 239.85972222222222]

In [65]:
reload(keras_functions)
debug = keras_functions.get_user_data(train_handles[i], binvars=binvars, month=3, maxtimepts=5000)
#debug[0][['ratingupdatetimeseconds', 'solvetimeseconds', 'starttimeseconds', 'stoptimeseconds', 'hours_solve_to_contest', 'hours_submit_to_solve']]
debug[0][['hours_solve_to_contest', 'hours_submit_to_solve']]

,hours_solve_to_contest,hours_submit_to_solve
0,240.279444,0.062500
23,243.030000,0.000000
24,265.554444,0.108889
27,219.193056,0.000000
28,264.398889,0.000000
30,270.010556,0.194722
32,157.605000,0.000000
34,241.484444,0.000000
35,156.241667,0.000000
36,195.294722,0.000000


In [71]:
maxt

46

In [ ]:
X_batch = np.concatenate(X, axis=0)

In [ ]:
X_batch.shape

In [ ]:
Y

In [ ]:
epochs = 5

In [ ]:
for e in range(epochs):
    